In [ ]:
!hostnamectl | grep "Operating System"

In [ ]:
!chmod +x ./utils/install_ffmpeg.sh
!sudo ./utils/install_ffmpeg.sh

In [ ]:
!pip install swig
!pip install stable-baselines3==2.0.0a5 gymnasium[box2d]
!pip install opencv-python

In [1]:
import cv2
import numpy as np
import gymnasium as gym
from utils.video_render import record_simulation

In [2]:
def random_action(env: gym.Env, obs: np.ndarray) -> int:
    return env.action_space.sample()

In [3]:
from IPython.display import Video

video_path = './videos/random.mp4'

record_simulation(random_action, video_path)
Video(video_path)

Video conversion successful: ./videos/random.mp4


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 661        |
|    ep_rew_mean          | 57.1       |
| time/                   |            |
|    fps                  | 1449       |
|    iterations           | 19         |
|    time_elapsed         | 214        |
|    total_timesteps      | 311296     |
| train/                  |            |
|    approx_kl            | 0.00773607 |
|    clip_fraction        | 0.0484     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.17      |
|    explained_variance   | 0.748      |
|    learning_rate        | 0.0003     |
|    loss                 | 43.7       |
|    n_updates            | 72         |
|    policy_gradient_loss | -0.0025    |
|    value_loss           | 183        |
----------------------------------------

In [4]:
from sagemaker.estimator import Estimator
import sagemaker

session = sagemaker.Session()

metric_definitions = [
    {'Name': 'ep_len_mean', 'Regex': 'ep_len_mean\s*\|\s*([\d\.]+)'},
    {'Name': 'ep_rew_mean', 'Regex': 'ep_rew_mean\s*\|\s*(-?[\d\.]+)'}
]

estimator = Estimator(
    role=,
    image_uri=,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    output_path=,
    session=session,
    hyperparameters = {
        'seed': 42,
        'total_timesteps': 1000000
    },
    metric_definitions = metric_definitions
)

training_job_name = 'training-lunar-lander'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
estimator.fit(job_name=training_job_name)

In [ ]:
training_job_name = 'training-lunar-lander-180424-7'
estimator.fit(job_name=training_job_name)

INFO:sagemaker:Creating training-job with name: training-lunar-lander-180424-7


2024-04-19 03:42:58 Starting - Starting the training job...
2024-04-19 03:43:14 Starting - Preparing the instances for training...
2024-04-19 03:43:50 Downloading - Downloading input data...
2024-04-19 03:44:10 Downloading - Downloading the training image.........
2024-04-19 03:45:35 Training - Training image download completed. Training in progress.2024-04-19 03:45:49,090 [INFO] Initialization of training script completed. Commencing environment setup.
2024-04-19 03:45:49,660 [INFO] CUDA is available. Utilizing GPU: Tesla T4
2024-04-19 03:45:50,625 [INFO] PPO model configured. Starting training process.
Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 88.3     |
|    ep_rew_mean     | -183     |
| time/              |          |
|    fps             | 3942     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------

In [5]:
import boto3
s3 = boto3.client('s3')
s3.download_file('aws-samples-avt', f'{training_job_name}/output/model.tar.gz', './artifacts/model.tar.gz')

In [6]:
!tar -xzf ./artifacts/model.tar.gz -C ./artifacts

tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


In [ ]:
from stable_baselines3 import PPO

model = PPO.load('./artifacts/model.zip')

In [9]:
action, _ = model.predict([-0.004, 1.403, -0.418, -0.317, 0.004, 0.094, 0., 0.])

action

array(3)

In [13]:
from utils.video_render import make_model_action

video_path = './videos/model.mp4'

model_action = make_model_action(model)
record_simulation(model_action, video_path)
Video(video_path)

Video conversion successful: ./videos/model.mp4
